In [ ]:
import pandas as pd
import tensorflow as tf 

In [ ]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
import re
#import enchant
import random

random.seed(1966)

In [ ]:
df_train=pd.read_csv('/content/drive/MyDrive/dataset/Home depot/home depot.zip', compression='zip',encoding='latin-1')
df_test=pd.read_csv('/content/drive/MyDrive/dataset/Home depot/homedepot-test.csv.zip', compression='zip',encoding='latin-1')
df_pro_desc=pd.read_csv('/content/drive/MyDrive/dataset/Home depot/product_descriptions.csv.zip',compression='zip')
df_attr=pd.read_csv('/content/drive/MyDrive/dataset/Home depot/attributes.csv.zip',compression='zip')

In [ ]:
df_brand=df_attr[df_attr.name=='MFG Brand Name'][["product_uid","value"]].rename(columns={"value":"brand"})

In [ ]:
df_brand

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
...,...,...
2044702,224422.0,Everbilt
2044734,224423.0,Coastal Shower Doors
2044747,224424.0,stufurhome
2044774,224425.0,Home Decorators Collection


In [ ]:
df_attr

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90° connection...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001.0,Bullet05,Made from 12-Gauge steel
...,...,...,...
2044798,224428.0,Pilot Bit Included,No
2044799,224428.0,Power Tool Product Type,Drill & Router Bits
2044800,224428.0,Shank Diameter,3/8 in
2044801,224428.0,Shank Style,Hex


In [ ]:
num_train=df_train.shape[0]


In [ ]:
num_train

74067

In [ ]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [ ]:
def str_stem(s):
  if isinstance(s,str):
    s = s.replace("inches","in.") # whole word
    s = s.replace("inch","in.") # whole word
    s = s.replace(" in ","in. ") # no period
    s = s.replace(" in.","in.") # prefix space
    #s = s.replace("''","ft.") # character
    s = s.replace(" feet ","ft. ") # whole word
    s = s.replace("feet","ft.") # whole word
    s = s.replace("foot","ft.") # whole word
    s = s.replace(" ft ","ft. ") # no period
    s = s.replace(" ft.","ft.") # prefix space
    s = s.replace(" pounds ","lb. ") # character
    s = s.replace(" pound ","lb. ") # whole word
    s = s.replace("pound","lb.") # whole word
    s = s.replace(" lb ","lb. ") # no period
    s = s.replace(" lb.","lb.")
    return "".join([stemmer.stem(re.sub('[^A-Za-z0-9-./]',' ',word)) for word in s.lower().split()])
  else:
    return "null"

In [ ]:
def str_common_word(str1,str2):
  str1,str2=str1.lower(),str2.lower()
  words,cnt=str1.split(),0
  for word in words:
    if str2.find(word)>=0:
      cnt+=1
  return cnt


In [ ]:
def str_whole_word(str1,str2,i_):
  str1,str2=str1.lower().strip(),str2.lower().strip()
  cnt=0
  while i_<len(str2):
    i_=str2.find(str1,i_)
    if i_==-1:
      return cnt
    else:
      cnt+=1
      i_+=len(str1)
  return cnt

In [ ]:
df_test

In [ ]:
df_all=pd.concat((df_train,df_test),axis=0,ignore_index=True)
df_all=pd.merge(df_all,df_pro_desc,how='left',on='product_uid')
df_all=pd.merge(df_all,df_brand,how='left',on='product_uid')

In [ ]:
df_train.columns

Index(['id', 'product_uid', 'product_title', 'search_term', 'relevance'], dtype='object')

In [ ]:
df_pro_desc.columns

Index(['product_uid', 'product_description'], dtype='object')

In [ ]:
df_all.head(4)

,id,product_uid,product_title,search_term,relevance,product_description,brand
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,Delta


In [ ]:
df_all.shape

(240760, 7)

In [ ]:
df_all=df_all[:4000]

In [ ]:
df_all['search_term']=df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title']=df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description']=df_all['product_description'].map(lambda x:str_stem(x))


In [ ]:
df_all['brand']=df_all['brand'].map(lambda x:str_stem(x))
df_all['len_of_query']=df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title']=df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)


In [ ]:
df_all['len_of_description']=df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['query_title_len_prop']=df_all['len_of_title']/df_all['len_of_query']
df_all['query_desc_len_prop']=df_all['len_of_description']/df_all['len_of_query']
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']

In [ ]:
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['product_title']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']


In [ ]:
df_all

,id,product_uid,product_title,search_term,relevance,product_description,brand,len_of_query,len_of_title,len_of_description,len_of_brand,product_info,query_in_title,query_in_description,word_in_title,word_in_description,query_title_len_prop,query_desc_len_prop,ratio_title,ratio_description,attr,word_in_brand,ratio_brand
0,2,100001,simpsonstrong-ti12-gaugangl,anglbracket,3.00,notonlidoanglmakejointstronger theyalsoprovidm...,simpsonstrong-ti,1,1,13,1,anglbracket\tsimpsonstrong-ti12-gaugangl\tnoto...,0,0,0,0,1.0,13.0,0.0,0.0,anglbracket\tsimpsonstrong-ti\tsimpsonstrong-t...,0,0.0
1,3,100001,simpsonstrong-ti12-gaugangl,lbracket,2.50,notonlidoanglmakejointstronger theyalsoprovidm...,simpsonstrong-ti,1,1,13,1,lbracket\tsimpsonstrong-ti12-gaugangl\tnotonli...,0,0,0,0,1.0,13.0,0.0,0.0,lbracket\tsimpsonstrong-ti\tsimpsonstrong-ti12...,0,0.0
2,9,100002,behrpremiumtexturdeckov1-gal. sc-141tugboatwoo...,deckover,3.00,behrpremiumtexturdeckovisaninnovsolidcolorcoat...,behrpremiumtexturdeckov,1,2,24,1,deckover\tbehrpremiumtexturdeckov1-gal. sc-141...,0,0,0,0,2.0,24.0,0.0,0.0,deckover\tbehrpremiumtexturdeckov\tbehrpremium...,0,0.0
3,16,100005,deltavero1-handlshoweronlifaucettrimkitin.chro...,rainshowerhead,2.33,updatyourbathroomwiththedeltaverosingle-handls...,delta,1,2,10,1,rainshowerhead\tdeltavero1-handlshoweronlifauc...,0,0,0,0,2.0,10.0,0.0,0.0,rainshowerhead\tdelta\tdeltavero1-handlshowero...,0,0.0
4,17,100005,deltavero1-handlshoweronlifaucettrimkitin.chro...,showeronlifaucet,2.67,updatyourbathroomwiththedeltaverosingle-handls...,delta,1,2,10,1,showeronlifaucet\tdeltavero1-handlshoweronlifa...,1,0,1,0,2.0,10.0,1.0,0.0,showeronlifaucet\tdelta\tdeltavero1-handlshowe...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,12425,102175,hamptonbay5-shelfstandardbookcasein.white,bookcaseshelv,3.00,thehamptonbaywhite5-shelfstandardbookcasisanat...,hamptonbay,1,1,4,1,bookcaseshelv\thamptonbay5-shelfstandardbookca...,0,0,0,0,1.0,4.0,0.0,0.0,bookcaseshelv\thamptonbay\thamptonbay5-shelfst...,0,0.0
3996,12429,102176,nantucketpaverberkshir13in.x19in.tanvariegirre...,belgiumblockpaver,2.00,thikitinclud60piecof19in.x13in.berkshirpatiobl...,nantucketpaver,1,2,13,1,belgiumblockpaver\tnantucketpaverberkshir13in....,0,0,0,0,2.0,13.0,0.0,0.0,belgiumblockpaver\tnantucketpaver\tnantucketpa...,0,0.0
3997,12432,102176,nantucketpaverberkshir13in.x19in.tanvariegirre...,paverstone,3.00,thikitinclud60piecof19in.x13in.berkshirpatiobl...,nantucketpaver,1,2,13,1,paverstone\tnantucketpaverberkshir13in.x19in.t...,0,0,0,0,2.0,13.0,0.0,0.0,paverstone\tnantucketpaver\tnantucketpaverberk...,0,0.0
3998,12433,102176,nantucketpaverberkshir13in.x19in.tanvariegirre...,pavebrick,2.67,thikitinclud60piecof19in.x13in.berkshirpatiobl...,nantucketpaver,1,2,13,1,pavebrick\tnantucketpaverberkshir13in.x19in.ta...,0,0,0,0,2.0,13.0,0.0,0.0,pavebrick\tnantucketpaver\tnantucketpaverberks...,0,0.0


In [ ]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info','attr','brand'],axis=1)
df_all.head()

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,query_in_title,query_in_description,word_in_title,word_in_description,query_title_len_prop,query_desc_len_prop,ratio_title,ratio_description,word_in_brand,ratio_brand
0,2,100001,3.00,1,1,13,1,0,0,0,0,1.0,13.0,0.0,0.0,0,0.0
1,3,100001,2.50,1,1,13,1,0,0,0,0,1.0,13.0,0.0,0.0,0,0.0
2,9,100002,3.00,1,2,24,1,0,0,0,0,2.0,24.0,0.0,0.0,0,0.0
3,16,100005,2.33,1,2,10,1,0,0,0,0,2.0,10.0,0.0,0.0,0,0.0
4,17,100005,2.67,1,2,10,1,1,0,1,0,2.0,10.0,1.0,0.0,0,0.0


In [ ]:
print (df_all[:10])
print("--- Features Set: %s minutes ---" % ((time.time() - start_time)/60))

   id  product_uid  relevance  ...  ratio_description  word_in_brand  ratio_brand
0   2       100001       3.00  ...                0.0              0          0.0
1   3       100001       2.50  ...                0.0              0          0.0
2   9       100002       3.00  ...                0.0              0          0.0
3  16       100005       2.33  ...                0.0              0          0.0
4  17       100005       2.67  ...                0.0              0          0.0
5  18       100006       3.00  ...                0.0              0          0.0
6  20       100006       2.67  ...                0.0              0          0.0
7  21       100006       3.00  ...                1.0              0          0.0
8  23       100007       2.67  ...                1.0              0          0.0
9  27       100009       3.00  ...                0.0              0          0.0

[10 rows x 17 columns]
--- Features Set: 51.77179535230001 minutes ---


In [ ]:
df_all.iloc[:2]

In [ ]:
num_train

74067

In [ ]:
df_train1=df_all.iloc[:3500]
df_test1=df_all.iloc[3500:]

In [ ]:
id_test=df_test1['id']
y_train=df_train1['relevance'].values
X_train=df_train1.drop(['id','relevance'],axis=1).values
X_test=df_test1.drop(['id','relevance'],axis=1).values


In [ ]:
RMSE= make_scorer(fmean_squared_error, greater_is_better=False)

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
clf = pipeline.Pipeline([('rfr', rfr)])


In [ ]:
param_grid = {'rfr__n_estimators' : [325],#list(range(109,110,1)), 
              'rfr__max_depth': [8], #list(range(7,8,1))
            }


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 10, verbose = 5, scoring=RMSE)
model.fit(X_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.8s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('rfr',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              max_samples=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                                                              min_samples_leaf=1,
                                                       

In [ ]:
model.best_params_

{'rfr__max_depth': 8, 'rfr__n_estimators': 325}

In [ ]:
model.best_score_

-0.5416349284000062

In [ ]:
y_pred = model.predict(X_test)
print(len(y_pred))
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission16.csv',index=False)
print("--- Training & Testing: %s minutes ---" % ((time.time() - start_time)/60))

500
--- Training & Testing: 68.23748751481374 minutes ---
